In [ ]:
import numpy as np
import torch

N = 1        # Number of system cases
l = 3        # Finite time horizon
n = 2        # Dimension of the state
G = 5        # Number of sensors in the ground set
K = 3        # Number of sensors to select at each time step
Q = list(range(1, G + 1))  # Sensor indices (1-based)
W = np.eye(n) * 0.4        # Process noise covariance
V = np.eye(G) * 0.7        # Measurement noise covariance
x0 = np.zeros((n, 1))      # Initial state (unused in this script)
S0 = np.eye(n)              # Initial covariance matrix

# What I gave below
# 1: linear system dynamics
# 2: sensor scheduling algorithm()


In [ ]:
C = np.random.rand(G, n)
A = np.random.rand(n, n)

# (process, measurement) noise terms should be implemented 

In [ ]:
scheduling_alg_dict = {'0':'random','1':'greedy','2':'randomized_greedy','3':'Brute_Force'}


if scheduling_alg == 0: # random
            Sigma, S = self.random_schedule(k,device = 'cuda')
        elif scheduling_alg == 1: # greedy
            Sigma, S = self.greedy_schedule(k, device = 'cuda'
        elif scheduling_alg == 2: # randomized_greedy
            Sigma, S = self.randomized_greedy_schedule(k, device = 'cuda')      
        elif scheduling_alg == 3: # Brute-Force
            Sigma_list, S_list, total_time_list = self.BF_schedule(z, days, device = 'cuda')
            return Sigma_list, S_list, total_time_list


        return S, np.trace(Sigma)

TypeError: object of type 'int' has no len()

In [ ]:
def random_schedule(self, k, device = 'cuda'):

        G = total_gauges
        selected_indices = np.random.choice(G, k, replace=False)
        S_rand = sorted(selected_indices)

        Sigma_rand = self.cov_matrix(S_rand, self.P, self.R, self.H, device)

        return Sigma_rand, S_rand

In [ ]:
def greedy_schedule(self, k, device = 'cuda'):
        G = total_gauges
        Q = list(range(1, G + 1))  # 1-based sensor indices

        # S_all = list(itertools.combinations(Q, k)) # All possible combinations of k sensors, like Q Choose k
        # need to change: every time selected the best ONE sensor from S, and then add it to S_g
        # until S_g has k sensors
        S_all = Q # All possible combinations of k sensors, like Q Choose k

        S_g = []

        # for S_cur in tqdm(S_all):
        while (len(S_g) < k):
            obj_cur_max = float('-inf')
            currentBestS = None
            # find the best next ONE sensor
            for S_cur in S_all:
                S_cur_list = [S_cur]
                Sigma_temp = self.cov_matrix(S_cur_list, self.P, self.R, self.H, device)
                # matrix multiplication? rather than inverse? 
                cur_trace = np.trace(Sigma_temp)
                
                marginal_gain = np.trace(self.P) - cur_trace
                # print(marginal_gain, "marginal gain")
                
                if abs(marginal_gain) > obj_cur_max:
                    obj_cur_max = marginal_gain
                    currentBestS = S_cur
            S_g.append(currentBestS)
            S_all.remove(currentBestS)
        Sigma_g = self.cov_matrix(S_g, self.P, self.R, self.H, device)
        return Sigma_g, S_g

In [ ]:
def randomized_greedy_schedule(self, k, device = 'cuda'):
        G = total_gauges
        Q = list(range(1, G + 1))  # 1-based sensor indices
        epsilon = 0.5
        # S_all = list(itertools.combinations(Q, k)) # All possible combinations of k sensors, like Q Choose k
        # need to change: every time selected the best ONE sensor from S, and then add it to S_g
        # until S_g has k sensors
        S_all = Q # All possible combinations of k sensors, like Q Choose k

        def sample_indices(G, Q, k, epsilon, selected):
            
            
            # Calculate the number of indices to sample
            s = int((G / k) * np.log(1 / epsilon))
            print("s!!!", s)
            if selected is None:
                available_indices = Q
            else: 
                available_indices = [item for item in Q if item not in selected]
            
            R = np.random.choice(available_indices, size=s)
            return R
    
        S_g = []

        # for S_cur in tqdm(S_all):
        while (len(S_g) < k):
            obj_cur_max = float('-inf')
            currentBestS = None
            # find the best next ONE sensor
            R = sample_indices(G, S_all, k, epsilon, currentBestS)
            for S_cur in R:
                print("R=", R)
                S_cur_list = [S_cur]
                Sigma_temp = self.cov_matrix(S_cur_list, self.P, self.R, self.H, device)
                
                cur_trace = np.trace(Sigma_temp)
                
                marginal_gain = np.trace(self.P) - cur_trace
               
                
                if abs(marginal_gain) > obj_cur_max:
                    obj_cur_max = marginal_gain
                    currentBestS = S_cur
    
            S_g.append(currentBestS)
            S_all.remove(currentBestS)
        Sigma_g = self.cov_matrix(S_g, self.P, self.R, self.H, device)
        return Sigma_g, S_g

In [ ]:
def BF_schedule(self, z, days, device = 'cuda'):
        
        Q = list(range(total_gauges))  # sensor index 0~ k-1
        S_all = list(itertools.combinations(Q, selected_gauges)) # All possible combinations of k sensors, like Q Choose k
        self.Sigma_set = [np.zeros((5175, 5175, len(S_all)**(i - 1))) for i in range(1, days + 2)] #Sigma_set[0].shape (n,n,1)
        S_g_list = []
        Sigma_g_list = [] 
        total_time_list = []
        total_time = 0
        self.Sigma_set[0][:,:,0] = self.P
        
        #self.Sigma_set = [np.zeros((5175, 5175, len(S_all)**i)) for i in range(days)] #Sigma_set[0].shape (n,n,1)

        for t in tqdm(range(1, days + 1)): # start from 1 

            # get current time
            curr_time = datetime.now()

            z[t-1] = z[t-1] - np.dot(self.S, np.dot(self.A0_day, self.Q0))
            obj_cur_min = np.inf
            diag_R = 0.01 * z[t-1] ** 2
            self.R = np.diag(diag_R)    
            best_idx = 0
            index_counter = 0  # Initialize global index counter
            cur_best_Sigma = None
            cur_best_S = None
            for j in range(len(S_all) ** (t-1)):
                
                for i, S_cur in enumerate(S_all):
                    Sigma_temp = self.cov_matrix(S_cur, self.Sigma_set[t-1][:, :, j], self.R, self.H, device)
                    
                    #idx = j * (len(S_all) ** (t - 1)) + i

                    idx = index_counter  # Use global counter as the index
                    index_counter += 1  # Increment the global counter

                    print(t, idx)
                    self.Sigma_set[t][:, :, idx] = Sigma_temp

                    obj_cur = np.trace(Sigma_temp)

                    if obj_cur < obj_cur_min:
                        obj_cur_min = obj_cur
                        cur_best_S = S_cur
                        cur_best_Sigma = Sigma_temp.copy()
                        best_idx = idx 
                        
            # get current time
            curr_time = datetime.now() - curr_time
            total_time += curr_time.total_seconds()
            total_time_list.append(total_time)
            print(f" Time taken for timestep {t}: {curr_time}")
        
        

        S_g_list.append(cur_best_S)
        Sigma_g_list.append(np.trace(cur_best_Sigma))

        a, b = divmod(best_idx, len(S_all))
        for t in range(days-1):
            Sigma_g_list.append(np.trace(self.Sigma_set[days - t - 1][:, :, a]))  # Add the previous Sigma
            best_idx = a
            a, b = divmod(best_idx, len(S_all))
            S_g_list.append(S_all[b])
            
        # For reverse
        for right in range(len(S_g_list) // 2):
            left = len(S_g_list) - right - 1

            temp1 = S_g_list[right]
            S_g_list[right] = S_g_list[left]
            S_g_list[left] = temp1

            temp2 = Sigma_g_list[right]
            Sigma_g_list[right] = Sigma_g_list[left]
            Sigma_g_list[left] = temp2
   

        return Sigma_g_list, S_g_list, total_time_list

1

In [ ]:
def cov_matrix(_, S, Sigma_prev, R, H, device = 'cuda'):
        result = None   
        k = len(S)
        def build_nonzeroH(H, S):
            S_sorted = sorted(S)
            H_hat = H[np.array(S_sorted) - 1, :]  # Convert to 0-based indexing
            return H_hat

        def build_nonzeroR(R, S):
            S_sorted = sorted(S)
            R_hat = R[np.ix_(np.array(S_sorted) - 1, np.array(S_sorted) - 1)]  # Convert to 0-based indexing
            return R_hat
        
        if device == 'cpu':
            
            if k == R.shape[0]:
                R_hat = R
                H_hat = H
            else:
                R_hat = build_nonzeroR(R, S)
                H_hat = build_nonzeroH(H, S)

            # one way to calculate P
            S = R_hat + np.dot(H_hat, np.dot(Sigma_prev, H_hat.T))
            K = np.dot(np.dot(Sigma_prev, H_hat.T), np.linalg.inv(S))
            Sigma_post = Sigma_prev - np.dot(np.dot(K,H_hat),Sigma_prev)
            
            # print("Sigma_post!")
            #self.x += np.dot(K, innovation)
            result = Sigma_post
        elif device == 'cuda':
            import torch

            os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
            os.environ["CUDA_VISIBLE_DEVICES"] = "1"

            if k == R.shape[0]:
                R_hat = R
                H_hat = H
            else:
                R_hat = build_nonzeroR(R, S)
                H_hat = build_nonzeroH(H, S)

            # one way to calculate P
            R_hat = torch.tensor(R_hat, device=device)
            H_hat = torch.tensor(H_hat, device=device)
            Sigma_prev = torch.tensor(Sigma_prev, device=device)
            
            S = R_hat + torch.matmul(H_hat, torch.matmul(Sigma_prev, H_hat.T))
            K = torch.matmul(torch.matmul(Sigma_prev, H_hat.T), torch.linalg.inv(S))
            Sigma_post = Sigma_prev - torch.matmul(torch.matmul(K, H_hat), Sigma_prev)
            result = Sigma_post.cpu().numpy()   
            # print("Sigma_post!")
            #self.x += np.dot(K, innovation)
            
            
        # another way to calculate P

        # Compute the inverse of (A * Sigma_prev * A^T + W)
        #try:
        #    inv_ASigmaA_T = np.linalg.inv(Sigma_prev)
        #except np.linalg.LinAlgError:
        #    # If singular, use pseudo-inverse
        #    inv_ASigmaA_T = np.linalg.pinv(Sigma_prev)

        ## Compute C_hat^T * V_hat * C_hat
        #HRH = H_hat.T @ R_hat @ H_hat

        ## Compute the inverse of (inv_ASigmaA_T + HRH)
        #try:
        #    Sigma = np.linalg.inv(inv_ASigmaA_T + HRH)
        #except np.linalg.LinAlgError:
        #    # If singular, use pseudo-inverse
        #    Sigma = np.linalg.pinv(inv_ASigmaA_T + HRH)

        return result